In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('processed_book_data.csv')

In [ ]:
df.drop(8,inplace=True)
df = df.iloc[:,1:]

In [3]:
def generate_embeddings(text):
    sentences = []
    sentences.append(sent_tokenize(text))
    sentences = [y for x in sentences for y in x]
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype = 'float32')
        word_embeddings[word] = coefs
    f.close()
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    return sentences, word_embeddings, clean_sentences   

In [4]:
def generate_vectors(word_embeddings, clean_sentences):
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    return sentence_vectors

In [5]:
# Finding Similarity to generate summary
def generate_similarity_matrix(sentence_vectors):
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    return sim_mat

In [27]:
# Using PageRank algorithm to find sort sentences based on most links (similarity)
book_summary = []
def generate_summary(sim_mat):
    summary = []
    #convert the sim_mat similarity matrix into the graph, the nodes in this graph will represent the sentences and the edges will represent 
    # the similarity scores between the sentences:
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph, max_iter = 600)
    # Summarizing text
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    for i in range(2):
        summary.append(ranked_sentences[i][1])
    sdf = pd.DataFrame(summary, columns=['summary'])
    final_summary = ' '.join(sdf['summary'])
    book_summary.append(final_summary)
    # return book_summary

In [26]:
df.reset_index(inplace=True)
df = df.iloc[:,1:]

In [28]:
for i in range(len(df)):
    sentences, word_embeddings, clean_sentences = generate_embeddings(df['Clean Text'].loc[i])
    sentence_vectors = generate_vectors(word_embeddings, clean_sentences)
    sim_mat = generate_similarity_matrix(sentence_vectors)
    generate_summary(sim_mat)

<ipython-input-3-65ed9f8ebb60>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
<ipython-input-3-65ed9f8ebb60>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
<ipython-input-3-65ed9f8ebb60>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
<ipython-input-3-65ed9f8ebb60>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
<ipython-input-3-65ed9f8ebb60>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [34]:
word_embeddings['rich']

array([-0.1328   ,  0.92261  ,  0.93544  , -0.63118  ,  0.50402  ,
       -0.30338  , -0.55406  ,  0.071319 , -0.13877  ,  0.11294  ,
       -0.30848  , -0.35173  ,  0.27514  , -0.015935 , -0.19586  ,
       -0.26719  ,  0.1023   , -1.127    , -0.35836  ,  0.80409  ,
       -0.12859  ,  0.15648  ,  0.57618  , -0.21437  , -0.029065 ,
        0.49854  , -0.27588  , -0.051191 , -0.23875  ,  0.24421  ,
        0.22476  ,  0.070125 ,  0.50378  , -1.1223   ,  0.4764   ,
       -0.06986  , -0.027617 ,  0.7853   , -0.68022  , -0.036379 ,
        0.37911  , -0.93087  , -0.011035 , -0.082644 , -0.27459  ,
        0.09938  , -0.26818  ,  0.29252  ,  0.62947  , -0.22741  ,
       -0.6275   ,  0.031109 , -0.054661 ,  0.16065  , -0.38845  ,
       -1.9376   , -0.10413  , -0.64834  ,  1.4837   , -0.13894  ,
       -0.049802 ,  0.55547  , -0.21925  , -0.90887  ,  1.4822   ,
       -0.6478   ,  1.1769   ,  0.1142   ,  0.33713  , -0.7661   ,
        0.37561  ,  0.0037597, -0.28029  ,  0.12944  , -0.1719

In [33]:
df['Summary'] = book_summary
book_summary[2][:500]

'” sometimes physically removing situation help maintain boundary you replenish physically emotionally spiritually given limit jesus often or remove get away danger put limit evil the bible urge u separate continue hurt u create safe place removing situation also cause one left behind experience loss fellowship may lead changed behavior matt 18:17–18 1 cor 5:11–13 01.boundaries 5/14/01 12:37 pm page 34 34 boundaries when relationship abusive many time way ﬁnally show person boundary real create s'

In [30]:
df

,Title,Author,Number of Pages,Text,Clean Text,Summary
0,Rich Dad Poor Dad,Robert Kiyosaki and Sharon Lechter,241,What The Rich Teach Their Kids About Money – \...,what the rich teach their kids about money – t...,” but one line i thought truthful it come scen...
1,An Imaginary Tale_ The Story of i the square r...,Paul J. Nahin,297,An Imaginary Tale\n\n An Imaginary Tale\n\n...,an imaginary tale an imaginary tale the story ...,speciﬁc limit partial fraction expan- sion sid...
2,"Boundaries_ When to Say Yes, How to Say No to ...","Henry Cloud, John Townsend",357,Resources by Henry Cloud and John Townsend\n\...,resources henry cloud john townsend boundaries...,” sometimes physically removing situation help...
3,Evil_ The Science Behind Humanity’s Dark Side,Julia Shaw,221,Also\tby\tDr.\tJulia\tShaw\n\nThe\tMemory\tIl...,also dr. julia shaw the memory illusion copyri...,“ sometimes one take action even one know mora...
4,Fallen_Angels_and_the_Origins_of_Evil_Elizabet...,Elizabeth Clare Prophet,455,\n\n1 \n\n \n\n \n\n \n\n2 \n\n \n\n \n\n \...,1 2 3 fallen angels and the origins ...,” 25 perhaps paul quote book enoch indirectly ...
5,Harry Potter ve Ateş Kadehi - J. K. Rowling,J.K. Rowling,912,Harry Potter\n\nve\n\nAteş Kadehi\n\nJ. K. Ro...,harry potter ateş kadehi j. k. rowling 2000 b...,t. düşünmüyor gibiydi viktor krum'la koyu bir ...
6,Living in the Light,Shakti Gawain,258,LIL Quark-FM 1/22/01 8:42 AM Page i\n\nLIV...,lil quark-fm 1/22/01 8:42 am page living the l...,” it ’ fulfillment ’ moment count in channel e...
7,See You At The Top,Zig Ziglar,386,I BELIEVE \n\nYou can get everything in li...,i believe you get everything life want help en...,tends create feeling inferiority when commenti...
8,Start Where You Are_ A Guide to Compassionate ...,Pema Chodron,242,This page intentionally left blank.\nThis pag...,this page intentionally left blank this page i...,” that turned major turning point life be- cau...
9,Subliminal Psychology 101_ How to Stealthily P...,Michael Pace,141,Subliminal Psychology 101\nHow to Stealthily ...,subliminal psychology 101 how stealthily penet...,” say three time anchoring you start train usi...


In [31]:
len(df['Clean Text'][0]), len(df['Summary'][0])

(250364, 13155)

In [32]:
# df.to_csv('Summary.csv', header=None)